## Inicialización

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from google.colab import drive

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras import layers

from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import SGD

random_state = 33

In [ ]:
#Para acceder a los ficheros de Google Drive
drive.mount('/content/drive')
# La carpeta datos debe estar en vuestro Drive, dentro de la carpeta 'Colab Notebooks'

#Experimento Mochila $N=20$ y $10^5$ casos

Debido al alto coste computacional de entrenar las redes neuronales, ejecutar etse experimento lleva una cantidad considerable de horas.

El problema utiliza $N$ objetos exactamente y los datos se cargan desde un fichero .csv como un vector de X casos donde cada posición es un vector de $4N+2$ variables que representan:   


- 0   Número de Objetos.  
- 1   Capacidad de la mochila.   
- 2 - 2N+1 Peso y Valor de cada objeto en ese orden.   
- 2N+2 - 3N+1 Vector binario solucion peor.  
- 3N+2 -4N+1 Vector binario de solucion mejor.



### Cargar datos

In [ ]:
file_path = 'drive/MyDrive/Colab Notebooks/datos/TFG/data_E1_Mochila.csv'

datos_Gr_FP = np.genfromtxt(file_path, delimiter=",") #Funcion para cargar datos desde txt, y en este caso desde csv

numItems = datos_Gr_FP[0][0]
x_Gr_FP = datos_Gr_FP[:, 1: int(3*numItems+2)]  # datos de entrada
y_Gr_FP = datos_Gr_FP[:, int(3*numItems+2):]   # datos de etiqueta
x_Gr_FP = np.array(x_Gr_FP, np.float64)
y_Gr_FP = np.array(y_Gr_FP, np.float64)

print("Número de objetos (N) =", numItems)
# Mostrar dimension del conjunto de muestras total
print("Forma de vector X de muestras:", x_Gr_FP.shape)
print("Forma de vector Y de etiquetas:", y_Gr_FP.shape)

print("Ejemplos:")
print("X = ",x_Gr_FP[0])
print("Y = ",y_Gr_FP[0])

### Separación de datos en Train y Test

Separo el conjunto de casos en test y train para evaluar los modelos utilizados. También se pueden utilizar funciones como `sklearn.model_selection.train_test_split()` pero de esta forma se pueden guardar los índices para realizar comprobaciones de forma más sencilla.

In [ ]:
np.random.seed(0)
trainPortion = 0.8 #porcentaje de train, el porcentaje de test será la resta de 1 menos el porcentaje de train

#-------------Obtener índices

indexesData = np.arange(len(y_Gr_FP)) #Indices del conjunto de muestras
#-------------Desordenar indices y separar en rain y test

np.random.shuffle(indexesData) #Desordenar indices de las muestras
numberTrain = round(len(indexesData)*trainPortion) #numero de muestras para train
trainIndexes = indexesData[:numberTrain]
testIndexes = indexesData[numberTrain:]

#-------------Datos desorden:ados para train y test

trainX_Gr_FP = x_Gr_FP[trainIndexes]
testX_Gr_FP = x_Gr_FP[testIndexes]
trainY_Gr_FP = y_Gr_FP[trainIndexes]
testY_Gr_FP = y_Gr_FP[testIndexes]

#-------------Copia de los datos para mantener datos originales después del preprocesado

original_trainX_Gr_FP = trainX_Gr_FP.copy()
original_testX_Gr_FP = testX_Gr_FP.copy()
original_trainY_Gr_FP = trainY_Gr_FP.copy()
original_testY_Gr_FP = testY_Gr_FP.copy()

#-------------Mostrar resultados

print('Muestras totales:  {}'.format(len(trainY_Gr_FP)+len(testX_Gr_FP)))
print('Muestras train:  {}'.format(len(trainY_Gr_FP)))
print('Muestras test:  {}'.format(len(testX_Gr_FP)))


## Preprocesamiento de los datos

Como los datos han sido generados artificialmente y no se han obtenido de fuentes externas, no es necesario realizar la búsqueda de datos faltantes o erroneos ni desbalanceo en las proporciones ya que los datos han sido generados teniendo en cuenta estos factores. La codificación de los datos también se ha tenido en cuenta en al generación de los datos.

### Comprobación de Duplicados en los datos

In [ ]:
# Comprobando duplicados en datos
unique_rows= np.unique(datos_Gr_FP, axis=0)
num_duplicates = datos_Gr_FP.shape[0] - unique_rows.shape[0]

print("Número de filas duplicadas en los datos:", num_duplicates)

### Escalar Datos

In [ ]:
# Crear el objeto MinMaxScaler
scaler = MinMaxScaler()

# Ajustar el escalador con los datos de entrenamiento y transformar tanto train como test
trainX_Gr_FP_scaled = scaler.fit_transform(trainX_Gr_FP)
testX_Gr_FP_scaled = scaler.transform(testX_Gr_FP)
trainX_Gr_FP_scaled = np.array(trainX_Gr_FP_scaled)
testX_Gr_FP_scaled = np.array(testX_Gr_FP_scaled)

print("X train = ",trainX_Gr_FP_scaled[0])
print("X test= ",testX_Gr_FP_scaled[0])

## Funciones y Métricas Fitness

In [ ]:
def valor_fitness(instancia, caso):
  instancia_local = instancia.copy()
  num_items = len(instancia)
  valores = caso[2:int(num_items*2+2):2]
  pesos = caso[1:int(num_items*2+1):2]
  pesoAct = np.sum(pesos*instancia_local)
  i=0
  while(pesoAct > caso[0]): # mientras el peso sea mayor
    if(instancia_local[i]):
      pesoAct -= pesos[i]
      instancia_local[i] = 0
    i += 1
  return np.sum(valores*instancia_local)


In [ ]:
def metrica_valor_fitness_entrada(y_true, y_pred, x):
  mejora = 0
  num_tot = len(x)
  num_items = len(y_true[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    valores = x[i][2:int(num_items*2+2):2]
    instancia_entrada = x[i][int(num_items*2+1):int(num_items*3+2)]
    valor_salida_pred = valor_fitness(predictions_binary[i], x[i])
    valor_entrada = np.sum(instancia_entrada * valores)
    mejora += valor_salida_pred / valor_entrada
  return mejora/num_tot

def metrica_valor_fitness_salida(y_true, y_pred, x):
  mejora = 0
  num_tot = len(x)
  num_items = len(y_true[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    valores = x[i][2:int(num_items*2+2):2]
    valor_salida_real = np.sum(y_true[i] * valores)
    valor_salida_pred = valor_fitness(predictions_binary[i], x[i])
    mejora += valor_salida_pred / valor_salida_real
  return mejora/num_tot

In [ ]:
def metrica_porcentaje_mejor_entrada(y_true, y_pred, x):
  num_aciertos = 0
  num_tot = len(x)
  num_items = len(y_true[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    valores = x[i][2:int(num_items*2+2):2]
    instancia_entrada = x[i][int(num_items*2+1):int(num_items*3+2)]
    valor_salida_pred = valor_fitness(predictions_binary[i], x[i])
    valor_entrada = np.sum(instancia_entrada * valores)
    if valor_salida_pred > valor_entrada:
      num_aciertos += 1
  return num_aciertos/num_tot

def metrica_porcentaje_mejor_salida(y_true, y_pred, x):
  num_aciertos = 0
  num_tot = len(x)
  num_items = len(y_true[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    valores = x[i][2:int(num_items*2+2):2]
    valor_salida_real = np.sum(y_true[i] * valores)
    valor_salida_pred = valor_fitness(predictions_binary[i], x[i])
    if valor_salida_pred > valor_salida_real:
      num_aciertos += 1
  return num_aciertos/num_tot

In [ ]:
print("Ratio de mejora de las soluciones FPTAS respecto de las soluciones aleatorias:",metrica_valor_fitness_entrada(trainY_Gr_FP,trainY_Gr_FP,trainX_Gr_FP)*100,"%")
print("Porcentaje de soluciones FPTAS mejores que las soluciones aleatorias:",metrica_porcentaje_mejor_entrada(trainY_Gr_FP,trainY_Gr_FP,trainX_Gr_FP)*100,"%")

## Creación del Modelo

Valores de entrada y salida de los modelos:

In [ ]:
input_shape  = 3*numItems +1
output_shape = numItems  #  Numero de valores a predecir (la solucion al problema)

Vamos a implementar una función que nos cree el modelo que vamos a entrenar y evaluar.

In [ ]:
def create_model(dense_neurons, input_shape, output_shape):
    local_model = keras.Sequential()

    # Capa de entrada
    local_model.add(layers.Input(shape=(int(input_shape),)))

    # capas ocultas con activación Relu
    for neurons in dense_neurons:
      local_model.add(layers.Dense(neurons, activation='relu'))

    # Capa de salida para clasificación con tantas neuronas como salidas esperadas
    local_model.add(layers.Dense(output_shape, activation='sigmoid'))
    return local_model

## Grid Search

Definimos la función para utilizar grid_search

In [ ]:
def single_grid_search(dense_neurons, input_shape, output_shape, X_scaled,X, Y, epochs, batch_size, learn_rate, _n_splits = 5, _random_state = 33):

    # Inicializar KFold
    kfold = KFold(n_splits=_n_splits, shuffle=True, random_state=_random_state)

    accuracy_results = []
    loss_results = []
    mejora_entrada_results = []
    mejora_salida_results = []
    porc_mejor_entrada_results = []
    porc_mejor_salida_results = []

    # Iterar sobre cada fold (validación cruzada)
    for train_index, test_index in kfold.split(X_scaled):
        # Dividir datos en entrenamiento y prueba
        trainX_scaled, valX_scaled = X_scaled[train_index], X_scaled[test_index]
        trainX, valX = X[train_index], X[test_index]
        trainY, valY = Y[train_index], Y[test_index]

        # Crear modelo
        local_model = create_model(dense_neurons, input_shape, output_shape)

        # Compilar el modelo
        sgd_optimizer = SGD(learning_rate=learn_rate)
        local_model.compile(optimizer=sgd_optimizer,
                            loss='binary_crossentropy',
                            metrics=['accuracy'])

        # Entrenar el modelo
        local_model.fit(
            trainX_scaled, trainY,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
        )

        # Evaluar el modelo
        scores = local_model.evaluate(valX_scaled, valY, verbose=0)
        loss_results.append(scores[0])
        accuracy_results.append(scores[1])

        # Evaluar metricas personalizadas
        y_pred = local_model.predict(valX_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_entrada_results.append(metrica_valor_fitness_entrada(valY,y_pred_binary,valX))
        mejora_salida_results.append(metrica_valor_fitness_salida(valY,y_pred_binary,valX))
        porc_mejor_entrada_results.append(metrica_porcentaje_mejor_entrada(valY,y_pred_binary,valX))
        porc_mejor_salida_results.append(metrica_porcentaje_mejor_salida(valY,y_pred_binary,valX))

    # Devolver el promedio de las evaluaciones
    return np.mean(loss_results), np.mean(accuracy_results),np.mean(mejora_entrada_results),np.mean(mejora_salida_results),np.mean(porc_mejor_entrada_results),np.mean(porc_mejor_salida_results), loss_results

Valores de los hiperparámetros a probar:

In [ ]:
# Valores de los hiperparámetros para el modelo base
dense_neurons  = (numItems*numItems,numItems*numItems,numItems*numItems/2,numItems*2)
epochs = 100
learning_rate = 0.1
batch_size = 64
k_folds = 2

# Valores de los  distintos hiperparámetros
learning_rate_search  = [0.1,0.01,0.001,0.0001,0.00001]
batch_size_search = [16,32,64,96,128]

> Hiperparámetro de la tasa de aprendizaje (learning rate)

In [ ]:
accuracy_mean_table = []
loss_mean_table = []
mejora_entrada_table = []
mejora_salida_table = []
porc_mejor_entrada_table = []
porc_mejor_salida_table = []

for lr in learning_rate_search:
    # Entrenar y evaluar
    results = single_grid_search(dense_neurons, input_shape, output_shape, trainX_Gr_FP_scaled, trainX_Gr_FP, trainY_Gr_FP, epochs, batch_size, lr, k_folds, random_state)

    loss_mean_table.append(results[0])
    accuracy_mean_table.append(results[1])
    mejora_entrada_table.append(results[2])
    mejora_salida_table.append(results[3])
    porc_mejor_entrada_table.append(results[4])
    porc_mejor_salida_table.append(results[5])


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Tasa de Aprendizaje': learning_rate_search,'Error de validación(función de pérdida)':loss_mean_table,'Accuracy':accuracy_mean_table,'Mejora Solucion Incial':mejora_entrada_table,
                           'Mejora Solucion Etiqueta':mejora_salida_table,'Porc Mejores que Inicial':porc_mejor_entrada_table, 'Porc Mejores que Etiqueta':porc_mejor_salida_table})
display(tableFrame)

> Hiperparámetro del tamaño de lote (batch size)

In [ ]:
accuracy_mean_table = []
loss_mean_table = []
mejora_entrada_table = []
mejora_salida_table = []
porc_mejor_entrada_table = []
porc_mejor_salida_table = []

for bs in batch_size_search:
    # Entrenar y evaluar
    results = single_grid_search(dense_neurons, input_shape, output_shape, trainX_Gr_FP_scaled, trainX_Gr_FP, trainY_Gr_FP, epochs, bs, learning_rate, k_folds, random_state)

    loss_mean_table.append(results[0])
    accuracy_mean_table.append(results[1])
    mejora_entrada_table.append(results[2])
    mejora_salida_table.append(results[3])
    porc_mejor_entrada_table.append(results[4])
    porc_mejor_salida_table.append(results[5])


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Tamaño de Lote': batch_size_search,'Error de validación(función de pérdida)':loss_mean_table,'Accuracy':accuracy_mean_table,'Mejora Solucion Incial':mejora_entrada_table,
                           'Mejora Solucion Etiqueta':mejora_salida_table,'Porc Mejores que Inicial':porc_mejor_entrada_table, 'Porc Mejores que Etiqueta':porc_mejor_salida_table})
display(tableFrame)

## Modelo Final

In [ ]:
# Valores de los hiperparámetros para el modelo final
dense_neurons  = (numItems*numItems,numItems*numItems,numItems*numItems/2,numItems*2)
batch_size = 64
epochs = 200
learning_rate = 0.1


final_model = create_model(dense_neurons, input_shape, output_shape)

# Compilar el modelo
sgd_optimizer = SGD(learning_rate=learning_rate)
final_model.compile(optimizer=sgd_optimizer,
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

# Entrenar el modelo
final_model.fit(
    trainX_Gr_FP_scaled, trainY_Gr_FP,
    epochs=epochs,
    batch_size=batch_size,
    verbose=0,
)

## Resultados

In [ ]:
accuracy_mean_table = []
mejora_entrada_table = []
mejora_salida_table = []
porc_mejor_entrada_table = []
porc_mejor_salida_table = []
loss_mean_table = []

# Evaluar el modelo
scores = final_model.evaluate(testX_Gr_FP_scaled, testY_Gr_FP, verbose=0)
loss_mean_table.append(scores[0])
accuracy_mean_table.append(scores[1])

# Evaluar metricas personalizadas
y_pred = final_model.predict(testX_Gr_FP_scaled,verbose=0)
umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)

mejora_entrada_table.append(metrica_valor_fitness_entrada(testY_Gr_FP, y_pred_binary, testX_Gr_FP))
mejora_salida_table.append(metrica_valor_fitness_salida(testY_Gr_FP, y_pred_binary, testX_Gr_FP))
porc_mejor_entrada_table.append(metrica_porcentaje_mejor_entrada(testY_Gr_FP, y_pred_binary, testX_Gr_FP))
porc_mejor_salida_table.append(metrica_porcentaje_mejor_salida(testY_Gr_FP, y_pred_binary, testX_Gr_FP))


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Error de test (función de pérdida)':loss_mean_table,'Accuracy':accuracy_mean_table,'Mejora Solucion Incial':mejora_entrada_table,
                           'Mejora Solucion Etiqueta':mejora_salida_table,'Porc Mejores que Inicial':porc_mejor_entrada_table, 'Porc Mejores que Etiqueta':porc_mejor_salida_table})
display(tableFrame)

## Curvas de Aprendizaje

In [ ]:
def create_learning_curves_mejora(curves_model, train_X_scaled, train_X, train_Y, val_X_scaled, val_X, val_Y, batch_size,epochs, iters):
    accuracy_results = [[],[]]
    loss_results = [[],[]]
    mejora_entrada_results = [[],[]]
    mejora_salida_results = [[],[]]
    porc_mejor_entrada_results = [[],[]]
    porc_mejor_salida_results = [[],[]]

    for i in range(iters):
        # Entrenar el modelo
        curves_model.fit(
            train_X_scaled, train_Y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=2,
        )

        e_type = 0 # Entrenameinto
        # Evaluar el modelo
        scores = curves_model.evaluate(train_X_scaled, train_Y, verbose=0)
        loss_results[e_type].append(scores[0])
        accuracy_results[e_type].append(scores[1])

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(train_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_entrada_results[e_type].append(metrica_valor_fitness_entrada(train_Y,y_pred_binary,train_X))
        mejora_salida_results[e_type].append(metrica_valor_fitness_salida(train_Y,y_pred_binary,train_X))
        porc_mejor_entrada_results[e_type].append(metrica_porcentaje_mejor_entrada(train_Y,y_pred_binary,train_X))
        porc_mejor_salida_results[e_type].append(metrica_porcentaje_mejor_salida(train_Y,y_pred_binary,train_X))


        e_type = 1 # Validación
        # Evaluar el modelo
        scores = curves_model.evaluate(val_X_scaled, val_Y, verbose=0)
        loss_results[e_type].append(scores[0])
        accuracy_results[e_type].append(scores[1])

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(val_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_entrada_results[e_type].append(metrica_valor_fitness_entrada(val_Y,y_pred_binary,val_X))
        mejora_salida_results[e_type].append(metrica_valor_fitness_salida(val_Y,y_pred_binary,val_X))
        porc_mejor_entrada_results[e_type].append(metrica_porcentaje_mejor_entrada(val_Y,y_pred_binary,val_X))
        porc_mejor_salida_results[e_type].append(metrica_porcentaje_mejor_salida(val_Y,y_pred_binary,val_X))

    return accuracy_results, loss_results, mejora_entrada_results, mejora_salida_results, porc_mejor_entrada_results, porc_mejor_salida_results


dense_neurons  = (numItems*numItems,numItems*numItems,numItems*numItems/2,numItems*2)
batch_size = 64
learning_rate = 0.1
curves_model = create_model(dense_neurons, input_shape, output_shape)

# Compilar el modelo
sgd_optimizer = SGD(learning_rate=learning_rate)
curves_model.compile(optimizer=sgd_optimizer,
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
epochs = 5
iters = 20
curves_results = create_learning_curves_mejora(curves_model, trainX_Gr_FP_scaled, trainX_Gr_FP, trainY_Gr_FP,testX_Gr_FP_scaled,testX_Gr_FP,testY_Gr_FP, batch_size, epochs,iters)

In [ ]:
accuracy_mean_table = []
mejora_entrada_table = []
mejora_salida_table = []
porc_mejor_entrada_table = []
porc_mejor_salida_table = []
loss_mean_table = []

# Evaluar el modelo
scores = curves_model.evaluate(testX_Gr_FP_scaled, testY_Gr_FP, verbose=0)
loss_mean_table.append(scores[0])
accuracy_mean_table.append(scores[1])

# Evaluar metricas personalizadas
y_pred = curves_model.predict(testX_Gr_FP_scaled,verbose=0)

umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)
mejora_entrada_table.append(metrica_valor_fitness_entrada(testY_Gr_FP,y_pred_binary,testX_Gr_FP))
mejora_salida_table.append(metrica_valor_fitness_salida(testY_Gr_FP,y_pred_binary,testX_Gr_FP))
porc_mejor_entrada_table.append(metrica_porcentaje_mejor_entrada(testY_Gr_FP,y_pred_binary,testX_Gr_FP))
porc_mejor_salida_table.append(metrica_porcentaje_mejor_salida(testY_Gr_FP,y_pred_binary,testX_Gr_FP))


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Error de validación medio (función de pérdida)':loss_mean_table,'Accuracy medio':accuracy_mean_table,'Mejora Solucion Incial':mejora_entrada_table,
                           'Mejora Solucion Etiqueta':mejora_salida_table,'Porc Mejores que Inicial':porc_mejor_entrada_table, 'Porc Mejores que Etiqueta':porc_mejor_salida_table})
display(tableFrame)

### Función de Pérdida

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Función de Pérdida",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Función de Pérdida",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[1][0], 'b',label=r'$Pérdida_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[1][1], 'orange',label=r'$Pérdida_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2)

plt.show()

### Accuracy

In [ ]:
plt.figure(figsize=(12, 7))
plt.title(r"Curva de Aprendizaje del $Accuracy$",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Accuracy",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[0][0], 'b',label=r'$Accuracy_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[0][1], 'orange',label=r'$Accuracy_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2)

plt.show()

### Ratio de mejora soluciones iniciales

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Mejora respecto de las Soluciones Iniciales",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Ratio de Mejora",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[2][0], 'b',label=r'$Mejora_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[2][1], 'orange',label=r'$Mejora_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2)

plt.show()

### Ratio de mejora soluciones Etiquetas

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Mejora respecto de las Etiquetas",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Ratio de Mejora",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[3][0], 'b',label=r'$Mejora_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[3][1], 'orange',label=r'$Mejora_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2)

plt.show()

### Porcentaje de soluciones mejores que iniciales

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje del Porcentaje de Mejores soluciones respecto de las Iniciales",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Porcentaje de Mejores soluciones",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), np.array(curves_results[4][0])*100, 'b',label=r'$Porcentaje_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), np.array(curves_results[4][1])*100, 'orange',label=r'$Porcentaje_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2)

plt.show()

### Porcentaje de soluciones mejores que Etiquetas

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje del Porcentaje de Mejores soluciones respecto de las Etiquetas",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Porcentaje de Mejores soluciones",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), np.array(curves_results[5][0])*100, 'b',label=r'$Porcentaje_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), np.array(curves_results[5][1])*100, 'orange',label=r'$Porcentaje_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2)

plt.show()

## Algoritmo Genético

#### Funciones de adaptación

In [ ]:
def transformar_instancia_mochila_directo(pesos_valores, peso_maximo):
    # Crear una lista que comienza con el peso máximo
    vector_transformado = [peso_maximo]

    # Añadir los pesos y valores alternados
    for peso, valor in pesos_valores:
        vector_transformado.append(peso)
        vector_transformado.append(valor)

    return vector_transformado


def transformar_instancia_mochila_inverso(vector_transformado):
    # Extraer el peso máximo, que es el primer elemento
    peso_maximo = vector_transformado[0]

    # Extraer los pesos y valores restantes
    pesos_valores = []
    # Iterar sobre el vector desde el segundo elemento hasta el final, tomando de dos en dos elementos
    for i in range(1, len(vector_transformado), 2):
        peso = vector_transformado[i]
        valor = vector_transformado[i + 1]
        pesos_valores.append([peso, valor])

    return peso_maximo, pesos_valores


def transformar_instancia_mochila_directo_con_sol(pesos_valores, peso_maximo,sol):
    # Crear una lista que comienza con el peso máximo
    vector_transformado = [peso_maximo]

    # Añadir los pesos y valores alternados
    for peso, valor in pesos_valores:
        vector_transformado.append(peso)
        vector_transformado.append(valor)
    for aux in sol:
        vector_transformado.append(aux)

    return vector_transformado


def concatenar_solucion_instancia(instancia,sol):
    # Crear una lista que comienza con el peso máximo
    vector_transformado = []

    # Añadir los pesos y valores alternados
    for aux in instancia:
        vector_transformado.append(aux)

    for aux in sol:
        vector_transformado.append(aux)

    return vector_transformado


def transformar_soluciones_no_factibles(sol, peso_max, pesos_valores):
  instancia_local = sol.copy()
  instancia_locacl = np.array(instancia_local)
  num_items = len(instancia_local)
  pesos = np.array([item[0] for item in pesos_valores])
  valores = np.array([item[1] for item in pesos_valores])
  #print(pesos,instancia_local)
  pesoAct = np.sum(pesos*instancia_local)
  i=0
  while(pesoAct > peso_max): # mientras el peso sea mayor
    if(instancia_local[i] == 1):
      pesoAct -= pesos[i]
      instancia_local[i] = 0
    i += 1
  #print(pesoAct, np.sum(pesos*instancia_local),peso_max)
  return instancia_local


#### Algoritmo Genético simple

In [ ]:
import random
from random import getrandbits, randint, choice

def fitness(sol, peso_max, pesos_valores):
    peso_total, valor_total = 0, 0
    for indice, valor in enumerate(sol):
        peso_total += (sol[indice] * pesos_valores[indice][0])
        valor_total += (sol[indice] * pesos_valores[indice][1])

    if (peso_max - peso_total) >= 0:
        return valor_total
    else: # Se descartan las soluciones no factibles
        return -1

def media_fitness(poblacion, peso_max, pesos_valores):
    sum_tot = sum(fitness(x, peso_max, pesos_valores) for x in poblacion if fitness(x, peso_max, pesos_valores) >= 0)
    return sum_tot / (len(poblacion) * 1.0)


def max_fitness(poblacion, peso_max, pesos_valores):
    valores_fitness = [fitness(x, peso_max, pesos_valores) for x in poblacion if fitness(x, peso_max, pesos_valores) >= 0]
    return max(valores_fitness) if valores_fitness else None

def generar_sol(numItems):
    return [ getrandbits(1) for x in range(numItems) ]

def generar_pobl(num_sols, numItems):
    return [ generar_sol(numItems) for x in range(num_sols) ]


def elegir_antecesor(valores, fitness_total, indice_a_ignorar): # Metodo de ruleta
    acumulado = 0
    valor_radnom = random.random()

    if indice_a_ignorar != -1:
        fitness_total -= valores[0][indice_a_ignorar]

    for indice, i in enumerate(valores[0]):
        if indice_a_ignorar == indice:
            continue
        acumulado += i
        if acumulado / fitness_total >= valor_radnom:
            return indice

def cruzar(soluciones):
    valores = list(zip(*soluciones))
    fitness_total = sum(valores[0])
    indice_antecesor1 = elegir_antecesor(valores, fitness_total, -1)
    indice_antecesor2 = elegir_antecesor(valores, fitness_total, indice_antecesor1)

    antecesor1 = valores[1][indice_antecesor1]
    antecesor2 = valores[1][indice_antecesor2]

    return antecesor1, antecesor2

def op_genetico(poblacion, peso_max, pesos_valores, num_sols_total, mut_prob=0.05):
    soluciones = [ [fitness(x, peso_max, pesos_valores), x] for x in poblacion if fitness(x, peso_max, pesos_valores) >= 0]
    soluciones.sort(reverse=True)


    nuevas_sols = []
    while len(nuevas_sols) < num_sols_total:
        antecesor1, antecesor2 = cruzar(soluciones)
        indice_mit = len(antecesor1) // 2
        nueva_sol = antecesor1[:indice_mit] + antecesor2[indice_mit:]
        nuevas_sols.append(nueva_sol)


    for sol_act in nuevas_sols:
        if mut_prob > random.random():
            pos_to_mutate = randint(0, len(sol_act)-1)
            if sol_act[pos_to_mutate] == 1:
                sol_act[pos_to_mutate] = 0
            else:
                sol_act[pos_to_mutate] = 1

    return nuevas_sols


In [ ]:
random.seed(1)

peso_max, pesos_valores = transformar_instancia_mochila_inverso(trainX_Gr_FP[0][:int(numItems*2+1)])
peso_max = int(peso_max)
print(peso_max,pesos_valores)

num_sols_total = 150
iters = 200
numItems = len(pesos_valores)
#print(numItems)

poblacion = generar_pobl(num_sols_total, numItems)
poblacion = [transformar_soluciones_no_factibles(pop,peso_max,pesos_valores) for pop in poblacion]


valores_medios = [media_fitness(poblacion, peso_max, pesos_valores)]
valores_max = [max_fitness(poblacion, peso_max, pesos_valores)]
for i in range(iters):
    poblacion = op_genetico(poblacion, peso_max, pesos_valores, num_sols_total)
    valores_medios.append(media_fitness(poblacion, peso_max, pesos_valores))
    valores_max.append(max_fitness(poblacion,peso_max , pesos_valores))



In [ ]:
plt.figure(figsize=(10, 6))
plt.title(r"Media del Valor $Fitness$ de la Población de Soluciones sin Red Neuronal",fontsize = 15)
plt.xlabel("Iteración",fontsize = 15)
plt.ylabel(r"Valor $fitness$ medio",fontsize = 15)
plt.ylim( 100000 , 450000)
plt.plot(range(iters+1), valores_medios,'b',label=r'$Fitness$ medio', linewidth=2)
plt.legend(fontsize = 15,loc = 'lower right')
plt.grid(visible=True,linewidth=0.2)
plt.show()

plt.figure(figsize=(10, 6))
plt.title(r"Valor $Fitness$ Máximo de la Población de Soluciones sin Red Neuronal",fontsize = 15)
plt.xlabel("Iteración",fontsize = 15)
plt.ylabel(r"Valor $fitness$ máximo",fontsize = 15)
plt.plot(range(iters+1), valores_max,'b',label=r'$Fitness$ máximo', linewidth=2)
plt.ylim( 200000 , 500000)



plt.legend(fontsize = 15,loc = 'lower right')
plt.grid(visible=True,linewidth=0.2)

plt.show()

#### Algoritmo Genético con red neuornal

In [ ]:
random.seed(1)

peso_max, pesos_valores = transformar_instancia_mochila_inverso(trainX_Gr_FP[0][:int(numItems*2+1)])
peso_max = int(peso_max)
print(peso_max,pesos_valores)

num_sols_total = 150
iters = 200
numItems = len(pesos_valores)

poblacion = generar_pobl(num_sols_total, numItems)
poblacion = [transformar_soluciones_no_factibles(pop,peso_max,pesos_valores) for pop in poblacion]


eps = 0.05
atasco = 0
red_prob = 0.1


valores_medios = [media_fitness(poblacion, peso_max, pesos_valores)]
valores_max = [max_fitness(poblacion, peso_max, pesos_valores)]
iters_modelo = []
print(valores_medios)
for i in range(iters):
    poblacion = op_genetico(poblacion, peso_max, pesos_valores, num_sols_total)
    valores_medios.append(media_fitness(poblacion, peso_max, pesos_valores))
    valores_max.append(max_fitness(poblacion, peso_max, pesos_valores))
    if valores_medios[-1] < (valores_medios[-2]*(1+eps)) :
        atasco += 1
        if atasco >= 20:
          iters_modelo.append(i)
          atasco = 0
          for sol_act in poblacion:
           if red_prob > random.random():
              y_pred = final_model.predict(np.array([concatenar_solucion_instancia(trainX_Gr_FP_scaled[0][:int(numItems*2+1)], sol_act)]),verbose=0)
              umbral = 0.5
              y_pred_binary = (y_pred >= umbral).astype(int)
              sol_fact = transformar_soluciones_no_factibles(y_pred_binary[0], peso_max, pesos_valores)
              sol_act = sol_fact.tolist()
    else: atasco = 0

In [ ]:
plt.figure(figsize=(10, 6))
plt.title(r"Media del Valor $Fitness$ de la Población de Soluciones con Red Neuronal",fontsize = 15)
plt.xlabel("Iteración",fontsize = 15)
plt.ylabel(r"Valor $fitness$ medio",fontsize = 15)
plt.plot(range(iters+1), valores_medios,'g',label=r'$Fitness$ Medio', linewidth=2)
plt.scatter(iters_modelo, [valores_medios[i] for i in iters_modelo], color='purple', label='Llamada a la Red Neuronal', s=100, zorder=5)
plt.ylim( 100000 , 450000)
plt.legend(fontsize = 15,loc = 'lower right')
plt.grid(visible=True,linewidth=0.2)

plt.show()



plt.figure(figsize=(10, 6))
plt.title(r"Valor $Fitness$ Máximo de la Población de Soluciones con Red Neuronal",fontsize = 15)
plt.xlabel("Iteración",fontsize = 15)
plt.ylabel(r"Valor $fitness$ máximo",fontsize = 15)
plt.plot(range(iters+1), valores_max,'g',label=r'$Fitness$ máximo', linewidth=2)
plt.scatter(iters_modelo, [valores_max[i] for i in iters_modelo], color='purple', label='Llamada a la Red Neuronal', s=100, zorder=5)
plt.ylim( 200000 , 500000)

plt.legend(fontsize = 15, loc = 'lower right')
plt.grid(visible=True,linewidth=0.2)

plt.show()

#### Ejemplos de Soluciones Generadas

Ejemplos de soluciones distintas generadas por la red para la misma instancia del problema pero con diferentes soluciones iniciales:

In [ ]:
sol_ini = transformar_soluciones_no_factibles(trainX_Gr_FP[0][-20:], peso_max, pesos_valores)
y_pred = final_model.predict(np.array([concatenar_solucion_instancia(trainX_Gr_FP_scaled[0][:int(numItems*2+1)],sol_ini )]),verbose=0)
umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)
sol_fact = transformar_soluciones_no_factibles(y_pred_binary[0], peso_max, pesos_valores)
sol_act = sol_fact.tolist()
print("Solución Inicial:",sol_ini, "Valor fitness", fitness(sol_ini, peso_max, pesos_valores))
print("Solución generada",sol_act, "Valor fitness", fitness(sol_act, peso_max, pesos_valores))

In [ ]:
sol_ini = transformar_soluciones_no_factibles(trainY_Gr_FP[1], peso_max, pesos_valores)
y_pred = final_model.predict(np.array([concatenar_solucion_instancia(trainX_Gr_FP_scaled[0][:int(numItems*2+1)],sol_ini )]),verbose=0)
umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)
sol_fact = transformar_soluciones_no_factibles(y_pred_binary[0], peso_max, pesos_valores)
sol_act = sol_fact.tolist()
print("Solución Inicial:",sol_ini, "Valor fitness", fitness(sol_ini, peso_max, pesos_valores))
print("Solución generada",sol_act, "Valor fitness", fitness(sol_act, peso_max, pesos_valores))

In [ ]:
sol_ini = transformar_soluciones_no_factibles(trainY_Gr_FP[3], peso_max, pesos_valores)
y_pred = final_model.predict(np.array([concatenar_solucion_instancia(trainX_Gr_FP_scaled[0][:int(numItems*2+1)],sol_ini )]),verbose=0)
umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)
sol_fact = transformar_soluciones_no_factibles(y_pred_binary[0], peso_max, pesos_valores)
sol_act = sol_fact.tolist()
print("Solución Inicial:",sol_ini, "Valor fitness", fitness(sol_ini, peso_max, pesos_valores))
print("Solución generada",sol_act, "Valor fitness", fitness(sol_act, peso_max, pesos_valores))